In [1]:
#Goal- take blurred images and predict what digit that image is.

import numpy as np # linear algebra
import pandas as pd # data processing
import matplotlib.pyplot as plt # pyplot is for showing images

In [2]:
data = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')

In [3]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
#splitting the data into training and validation data
data = np.array(data)
m , n = data.shape             #m - amount of rows , n - amounts of colmnuns(features)+1
np.random.shuffle(data)        # shuffle before splitting into dev and training sets

data_val = data[1:1000].T
y_dev = data_val[0]
x_dev = data_val[1:n]

data_train = data[1:1000].T
y_train = data_train[0]
x_train = data_train[1:n]



In [5]:
x_train[:,0].shape

(784,)